### diabetics prediction 

In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv("./Diabetes_XTrain.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,7,168,88,42,321,38.2,0.787,40
1,8,110,76,0,0,27.8,0.237,58
2,7,147,76,0,0,39.4,0.257,43
3,2,100,66,20,90,32.9,0.867,28
4,4,129,86,20,270,35.1,0.231,23


In [15]:
df.shape

(576, 8)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               576 non-null    int64  
 1   Glucose                   576 non-null    int64  
 2   BloodPressure             576 non-null    int64  
 3   SkinThickness             576 non-null    int64  
 4   Insulin                   576 non-null    int64  
 5   BMI                       576 non-null    float64
 6   DiabetesPedigreeFunction  576 non-null    float64
 7   Age                       576 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 36.1 KB


In [36]:
df_train=pd.read_csv("./Diabetes_YTrain.csv")

In [22]:
df_train.head()

,Outcome
0,1
1,0
2,1
3,1
4,0


In [23]:
df_train.shape

(576, 1)

In [10]:
for col in df.columns:
    print(f"{col} : {(df[col]==0).sum()}")

Pregnancies : 81
Glucose : 3
BloodPressure : 29
SkinThickness : 171
Insulin : 278
BMI : 9
DiabetesPedigreeFunction : 0
Age : 0


In [30]:
df=df.drop(['SkinThickness','Insulin'],axis=1)

In [32]:
df.head(2)

,Pregnancies,Glucose,BloodPressure,BMI,DiabetesPedigreeFunction,Age
0,7,168,88,38.2,0.787,40
1,8,110,76,27.8,0.237,58


In [33]:
df.shape

(576, 6)

In [37]:
df_test=pd.read_csv("./Diabetes_Xtest.csv")

In [38]:
df_test.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0,180,90,26,90,36.5,0.314,35
1,2,93,64,32,160,38.0,0.674,23


In [39]:
df_test.shape

(192, 8)

In [40]:
df_test=df_test.drop(['SkinThickness','Insulin'],axis=1)

### Replacing 0 with mean

In [53]:
# zeros in X_train
for col in df.columns:
    print(f"{col} : {(df[col]==0).sum()}")

Pregnancies : 81
Glucose : 3
BloodPressure : 29
BMI : 9
DiabetesPedigreeFunction : 0
Age : 0


In [54]:
# zeroes in X_test
for col in df_test.columns:
    print(f"{col} : {(df_test[col]==0).sum()}")

Pregnancies : 30
Glucose : 2
BloodPressure : 6
BMI : 2
DiabetesPedigreeFunction : 0
Age : 0


In [61]:
# X_train
for col in df.columns:
    for i in range(df[col].shape[0]):
        if df[col].loc[i]==0:
            df[col].loc[i]=df[col].mean()
            
# X_test
for col in df_test.columns:
    for i in range(df_test[col].shape[0]):
        if df_test[col].loc[i]==0:
            df_test[col].loc[i]=df_test[col].mean()

In [63]:
# zeros in X_train
for col in df.columns:
    print(f"{col} : {(df[col]==0).sum()}")
print()    
# zeroes in X_test
for col in df_test.columns:
    print(f"{col} : {(df_test[col]==0).sum()}")

Pregnancies : 0
Glucose : 0
BloodPressure : 0
BMI : 0
DiabetesPedigreeFunction : 0
Age : 0

Pregnancies : 0
Glucose : 0
BloodPressure : 0
BMI : 0
DiabetesPedigreeFunction : 0
Age : 0


In [64]:
X_train=df.values
Y_train=df_train.values
X_test=df_test.values

### K-NN Algorithm

In [142]:
def distance(x1,x2):
    return np.sqrt(sum((x1-x2)**2))

def KNN(X,Y,query_point,k=10):
    
    dist=[]
    m=X.shape[0]
    
    for i in range(m):
        dist.append((distance(X[i],query_point),Y[i]))
    dist=sorted(dist)
    
    dist=dist[:k+1]
    
    dist=np.array(dist)
    
    new_val=np.unique(dist[:,1],return_counts=True)
    
    prediction=np.argmax(new_val[1])
    
    return prediction
    

In [143]:
prediction=[]
for i in range(X_test.shape[0]):
    prediction.append(KNN(X_train,Y_train,X_test[i]))

In [144]:
predict=pd.DataFrame({'Outcome':prediction})
predict.to_csv("KNN_Scratch.csv",index=False)

### 78% Accuracy

### Using SVM Inbuilt

In [80]:
from sklearn import svm

In [97]:
svm_=svm.SVC(kernel='poly')

In [98]:
svm_.fit(X_train,Y_train)

C:\Users\SS\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [99]:
pred_svm=svm_.predict(X_test)

In [100]:
predict1=pd.DataFrame({'Outcome':pred_svm})
predict1.to_csv("SVM_inbuilt.csv",index=False)  # 77% accuracy

### Logistic regression

In [90]:
from sklearn.linear_model import LogisticRegression

In [133]:
lr=LogisticRegression(max_iter=50)

In [92]:
lr.fit(X_train,Y_train)

C:\Users\SS\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [95]:
predict2=lr.predict(X_test)

In [96]:
predict=pd.DataFrame({'Outcome':predict2})
predict.to_csv("Logistic regression.csv",index=False) # 77% Accuracy

In [121]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

In [105]:
knn=KNeighborsClassifier()

In [106]:
rf=RandomForestClassifier()

In [102]:
import multiprocessing

In [113]:
cpus=multiprocessing.cpu_count()
print(cpus)

8


In [134]:
names = [
          "SVM",
         "Logistic Regression",
         "Random Forest",
         ]

classifiers = [
    svm.SVC(),
    lr,
    rf
    ]


params = [
    {
        'regressor':[svm.SVC()],
        'regressor__kernel':['linear','rbf','poly','sigmoid'],
        'regressor__C':[0.1,0.2,0.5,1.0,2.0,5.0]
    },
    
    {
        'regressor':[lr],
        'regressor__C':[0.1,0.2,0.5,1.0,2.0,5.0]
        
    }
    ,
    {
        'regressor':[knn],
        'regressor__n_neighbors':[10,12,15,20,25]
        
    }
    ,
    {
        'regressor':[rf],
        'regressor__criterion':['entropy','gini'],
        'regressor__max__depth':[10,14,16,18]
        
    }
        
]

In [135]:
for name, classifier, params in zip(names, classifiers, params):
    clf_pipe = Pipeline([
        ('regressor', classifier),
    ])
    gs_clf = GridSearchCV(clf_pipe, param_grid=params, n_jobs=cpus)
    clf = gs_clf.fit(X_train, Y_train)
    

C:\Users\SS\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\SS\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\SS\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  

In [136]:
gs_clf.best_estimator_

Pipeline(memory=None,
         steps=[('regressor',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=20, p=2,
                                      weights='uniform'))],
         verbose=False)

In [138]:
gs_clf.best_score_

0.7605247376311844